1. API Setup

In [0]:
%python
import os
import requests
import time

# Alpha Vantage Configuration
ALPHA_VANTAGE_API_KEY =  os.getenv('MY_API_KEY')
SYMBOL = "AAPL"
INTERVAL = "1min"  # Possible values: 1min, 5min, 15min, 30min, 60min
ALPHA_VANTAGE_API_URL = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={SYMBOL}&interval={INTERVAL}&apikey={ALPHA_VANTAGE_API_KEY}"


 # Kafka setup

In [0]:
%python
%pip install confluent_kafka

  Obtaining dependency information for confluent_kafka from https://files.pythonhosted.org/packages/e0/37/4ab00b1562ba8f9b7a4e91fde2ce8b6105e9901a05f6852ffaf2fec44417/confluent_kafka-2.5.3-cp311-cp311-manylinux_2_28_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/3.9 MB ? eta -:--:--
   ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/3.9 MB 1.8 MB/s eta 0:00:03
   ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/3.9 MB 3.7 MB/s eta 0:00:01
   ━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.9/3.9 MB 9.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━ 2.4/3.9 MB 17.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 3.9/3.9 MB 25.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 21.4 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

In [0]:
%python
kafka_bootstrap_servers = os.getenv('MY_KAFKA_SERVER')
kafka_topic = os.getenv('MY_KAFKA_TOPIC')


# Kafka configuration and pulling data from API to Kafka topic

In [0]:

# Import necessary libraries
from confluent_kafka import Producer
import requests
import json
import time

# Kafka Configuration
conf = {
    'bootstrap.servers': 'pkc-56d1g.eastus.azure.confluent.cloud:9092',
    'client.id': 'alpha-vantage-producer',
    'security.protocol': 'SASL_SSL',
    'sasl.mechanism': 'PLAIN',
    'sasl.username': 'OPJG2TBO5A6V3KGQ',
    'sasl.password': 'NktHxsYoNZZjBNQyfyBMsfqxo3FIe1soT6VhBiL82np1pTQtmqJwYvCzQTRULLVk'
}

# Initialize Kafka producer
producer = Producer(conf)

def delivery_report(err, msg):
    """Callback called once the message is delivered or delivery failed."""
    if err:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [{msg.partition()}]")

def fetch_and_send_stock_data():
    # Fetch stock data from Alpha Vantage API
    response = requests.get(ALPHA_VANTAGE_API_URL)
    
    if response.status_code == 200:
        stock_data = response.json()
        # Extract time series data
        time_series = stock_data.get("Time Series (1min)", {})
        for timestamp, data in time_series.items():
            message = {
                "symbol": SYMBOL,
                "timestamp": timestamp,
                "open": data["1. open"],
                "high": data["2. high"],
                "low": data["3. low"],
                "close": data["4. close"],
                "volume": data["5. volume"]
            }
            # Send each data point to Kafka
            producer.produce(
                kafka_topic, 
                key=SYMBOL, 
                value=json.dumps(message), 
                callback=delivery_report
            )
            producer.flush()
    else:
        print(f"Failed to fetch stock data: {response.status_code}")

# Call this function periodically (respecting Alpha Vantage rate limits)
while True:
    fetch_and_send_stock_data()
    time.sleep(60)  # Sleep to respect the API rate limit (1 minute)

Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_d

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can